## Convert ONNX Model to TensorFlow

This tutorial loads the MNIST ONNX model generated from *PyTorch to TF.ipynb* and saves within the TensorFlow framework. 

The original tutorial utilizes the *super_resolution.onnx* model and performs processing on the image by resizing, eventually saving the resized image. 

Reference: 
https://github.com/onnx/tutorials/blob/master/tutorials/OnnxTensorflowImport.ipynb

**Workflow**

0. Ignore warning messages
1. Load ONNX file, mnist-pyt.onnx
2. Import ONNX model to TensorFlow

Note: tf_rep is a python class containing four members: graph, inputs, outputs, and tensor_dict 

In [1]:
import onnx
import warnings
from onnx_tf.backend import prepare

warnings.filterwarnings('ignore')

modelIn = "mnist-pyt.onnx"
modelOut = "mnist-pyt.pb"

onnx_model = onnx.load(modelIn)
tf_rep = prepare(onnx_model)
tf_rep.export_graph(modelOut)

print("Model folder created: ", modelOut)

INFO:tensorflow:Assets written to: mnist-pyt.pb\assets
Model folder created:  mnist-pyt.pb


In [2]:
# Use Netron to view model
import netron

modelLoad = 'mnist-pyt.pb\saved_model.pb'

# ONNX Model
netron.start(modelIn, port=8081)

# Tensorflow Model
#netron.start(modelLoad, port=8082)

Serving 'mnist-pyt.onnx' at http://localhost:8081

Stopping http://localhost:8081
Serving 'mnist-pyt.pb\saved_model.pb' at http://localhost:8081


In [3]:
# Print tf_rep nodes:
print("Model input nodes: ", tf_rep.inputs)
print("Model output nodes: ", tf_rep.outputs)

print("\n All Model nodes: ", tf_rep.tensor_dict)

Model input nodes:  ['input.1']
Model output nodes:  ['20']

 All Model nodes:  None


### Prepare Input for Inference

Warning: This code is intended to run using TensorFlow 2.2.0

In [4]:
import tensorflow as tf
import tensorboard as tb
import numpy as np
import cv2

import logging, os

logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
 
INPUT_TENSOR_NAME = 'input.1:0'
OUTPUT_TENSOR_NAME = 'add_4:0'
IMAGE_PATH="img_1.jpg"
PB_PATH=modelOut
 
img = cv2.imread(IMAGE_PATH)
img = np.dot(img[...,:3], [0.299, 0.587, 0.114])
img = cv2.resize(img, dsize=(28, 28), interpolation=cv2.INTER_AREA)
img.resize((1, 1, 28, 28))

In [16]:
import numpy as np
from IPython.display import display
from PIL import Image

image = "img_1.jpg"

img = Image.open(image).resize((28, 28))
display(img) # show the image
img_ycbcr = img.convert("YCbCr")
img_y, img_cb, img_cr = img_ycbcr.split()
model_y = np.asarray(img_y, dtype=np.float32)[np.newaxis, np.newaxis, :, :]

### Run Model withing TensorFlow

In [17]:
# Run model and print shape
mod_image = tf_rep.run(model_y)._0
print(mod_image.shape)

(1, 10)
